In [56]:
import pandas as pd
import codecs

##Import section to ensure the main directory is in the path
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

## Importing from the main directory now
from dbf900_main import parse_record
from dbf900_layouts import dbf900_layout

In [57]:
##From https://github.com/skylerbast/TxRRC_data
##Generates the block of bytes from the file.
def yield_blocks(file, n):
    block_bytes = file.read(n)
    while block_bytes:
        yield block_bytes
        block_bytes = file.read(n)

In [58]:
##From https://github.com/skylerbast/TxRRC_data
##converts bytes to string through codecs decoding
def display(data):
    ebcdic_decoder = codecs.getdecoder('cp1140')
    decoded = ebcdic_decoder(data)
    val = decoded[0]
    
    return val

In [59]:
file_path = r'C:\PublicData\Texas\TXRRC\index\dbf900.ebc' ##Local storage location
##file origin: ftp://ftpe.rrc.texas.gov/shfwba/dbf900.ebc.gz

file = open(file_path, 'rb') ##Opens the .ebc file and reads it as bytes

split_records = []  ## array for records
n  = 247 ##Number of bytes for each record in dbf900.ebc file

In [60]:
well_ct = 0 ##counter for number of wells 
for block in yield_blocks(file, n): ##for each 
    startval = display(block[0:2]) ##
    if startval == '01': ##check for wbroot record
        well_ct+=1 ##increase well count by one
        API = display(block[2:10]) ##api value in records '01'
    
    layout = dbf900_layout(startval)['layout'] ##Retrieves layout for the block
    
    ##reading the records needs to be in bytes and not decode the whole block before parsing
    ##requires change to dbf900_formats to handle incoming bytes
    parsed_vals = parse_record(display(block), layout)  ##Decodes the block based on layout
    
    split_records.append([API,record])  ##adds record to the array
    
    if well_ct >= 100: ##limiting number of wells used in example
        break

In [61]:
split_records

[['00100001',
  {'RRC-TAPE-RECORD-ID': '01',
   'WELL-BORE-API-ROOT': '00100001',
   'WB-NXT-AVAIL-SUFFIX': '01',
   'WB-NXT-AVAIL-HOLE-CHGE-NBR': '01',
   'WB-FIELD-DISTRICT': '06',
   'WB-RES-CNTY-CODE': '001',
   'WB-ORIG-COMPL-DATE': '10/27/1963',
   'WB-TOTAL-DEPTH': 0,
   'WB-VALID-FLUID-LEVEL': 0,
   'WB-CERTIFICATION-REVOKED-DATE': None,
   'WB-CERTIFICATION-DENIAL-DATE': None,
   'WB-DENIAL-REASON-FLAG': '0',
   'WB-ERROR-API-ASSIGN-CODE': '',
   'WB-REFER-CORRECT-API-NBR': '00000000',
   'WB-DUMMY-API-NUMBER': '00100001',
   'WB-DATE-DUMMY-REPLACED': None,
   'WB-NEWEST-DRL-PMT-NBR': 0,
   'WB-CANCEL-EXPIRE-CODE': '',
   'WB-EXCEPT-13-A': 'N',
   'WB-FRESH-WATER-FLAG': 'N',
   'WB-PLUG-FLAG': 'N',
   'WB-PREVIOUS-API-NBR': '00000000',
   'WB-COMPLETION-DATA-IND': 'Y',
   'WB-HIST-DATE-SOURCE-FLAG': 0,
   'WB-EX14B2-COUNT': 13,
   'WB-DESIGNATION-HB-1975-FLAG': 'A',
   'WB-DESIGNATION-EFFECTIVE-DATE': '09/01/1993',
   'WB-DESIGNATION-REVISED-DATE': None,
   'WB-DESIGNATION-LET